In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool,PDFSearchTool
from crewai import LLM


In [ ]:

search_tool = SerperDevTool(n_results=3)

In [ ]:


# Create an agent with all available parameters
agent = Agent(
    role="Topic Analysist",
    goal="Using the tool that i gave to you analyze and interpret complex datasets to provide actionable insights",
    backstory="With over 10 years of experience in data science and machine learning, "
              "you excel at finding patterns in complex datasets.",
    llm=ollama_llm, 
    tools=[SerperDevTool()],
    verbose=True
)

In [ ]:
result = agent.kickoff("Chi è Kenan Yildiz")

print(result.raw)

 Provare in autonomia a creare una crew con un solo agente ed un solo task, con crewAI in un file .py/.ipynb con la configurazione che si preferisce, in modo che esegua la ricerca a piacere su di un file PDF (https://www.its-ictacademy.com/wp-content/uploads/2024/10/Regolamento-Iscrizioni-1.66fd1de210a445.09813304.pdf) tramite il tool PDFSearchTool configurato con llm ed embedder locale.


In [ ]:
ollama_llm= LLM(model="ollama/qwen3:8b",base_url="http://localhost:11434")



In [ ]:
tool = PDFSearchTool(
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="bge-m3:latest",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/bge-m3:latest",
                # title="Embeddings",
            ),
        ),
    )
)

## Es 2


In [ ]:
from crewai import Agent, Crew, Task, Process

agent_one = Agent (
            role="Researcher",
            goal="Synopsis specialist, expert, actors and details about the {film}",
            backstory="A diligent researcher with a keen eye for detail",
            llm=ollama_llm,
            verbose=True,
            tools=[SerperDevTool()]
        )

agent_two = Agent(
            role="Cinematographic critique",
            goal="Expert about the analysis and rewiew about {film}",
            backstory="A diligent researcher with a keen eye for detail",
            llm=ollama_llm,
            verbose=True
        )

task_1 = Task(
            description="Search the informations about the movie, informations about actors, and very deep details and trivia's about the movie",
            expected_output="A text with the summarized informations obtained in the research",
            agent=agent_one
        )

task_2 = Task(
            description="Use the output of the Researcher to create a review about the movie in question",
            expected_output="A text with the review of the film, analyzed in details and with the focus on what could be improved",
            agent=agent_two
        )




In [ ]:
crew = Crew(
            agents=[agent_one, agent_two],
            tasks=[task_1, task_2],
            process=Process.sequential,
            verbose=True
        )
    

In [ ]:
input_films = {"film":"Inception"}

result = crew.kickoff(inputs=input_films)

print(result.raw)

## ES 3


In [ ]:
ollama_llm= LLM(model="ollama/qwen3:8b",base_url="http://localhost:11434")

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    llm=ollama_llm,
    allow_code_execution=False,
)

# Define your agents
developer = Agent(
    role="Developer",
    goal="Writing an efficent and computational efficent code line, without ",
    backstory="You're an expert programmer, specialized in various programming language.",
    llm=ollama_llm,
    allow_code_execution=False
)

tester = Agent(
    role="Tester",
    goal="identifying the errors in the code line,without letting the code go in error.",
    backstory="You are an old programmer, experienced in your task. You are one of the best code reviewers in the company",
    llm=ollama_llm,
    allow_code_execution=False
)

# Define your task
task_1 = Task(
    description="delegating the other agents in writing the code task in an efficent way",
    expected_output="a full workflow with the other agents,and also a full line of code without any error",
    agent=manager   
)

task_2 = Task(
    description="Writing the {code} at the given input",
    expected_output="some full code lines written by the agents, without any errors. The line of codes should run without any interruption",
    agent=developer   
)


task_3= Task(
    description="the agent should review the code from the developer agent to search for any errors and fix them",
    expected_output="it should return a full line of code without any errors, and some advices to increase the performances",
    agent=tester
)





# Instantiate your crew with a custom manager
crew = Crew(
    agents=[developer, tester],
    tasks=[task_3,task_2,task_1],
    manager_agent=manager,
    process=Process.hierarchical,
    verbose=True
)


In [ ]:
input_code = {"code":"a program that sums the first 10 numbers between 1 an 10"}

result = crew.kickoff(inputs=input_code)

print(result)

## Es Knowledge


In [ ]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

In [ ]:

content1 = "Users name is Andrea. He is 24 years old and lives in Civitavecchia."
string_source = StringKnowledgeSource(content=content1)

In [ ]:
ollama_llm= LLM(model="ollama/bge-m3",base_url="http://localhost:11434")

In [ ]:
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="You are a master at understanding people and their preferences.",
    verbose=True,
    allow_delegation=False,
    llm=ollama_llm,
    embedder={
        "provider":"ollama", "url":"http://localhost:11434/api/embeddings",'model_name':"bge-m3"
    }
)


task = Task(
    description="Answer the following question {question}",
    expected_output="An answer to the question",
    agent=agent
    
)


crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here
)


In [ ]:
result = crew.kickoff(inputs={"question": "What city does Andrea live in and how old is he?"})

## Es 4

In [ ]:
from crewai import Agent,Task,Crew
from crewai_tools import SerperDevTool
from crewai import LLM,Process

ollama_llm= LLM(model="ollama/qwen3:0.6b",base_url="http://localhost:11434")

In [ ]:
tool = SerperDevTool()

In [ ]:
logic = Agent(
    role="logic",
    goal="You must analyze the problem and find a logical solution for it, you must work in a logical and direct way",
    backstory="You solved thousand of logical problems,you are an expert in this field and find amusing how simple these logical problems can be,everyone trust your judgment, but you also rely on others to find a collective solution",
    allow_delegation=True,
    tools=[SerperDevTool()]

)

lathink = Agent(
    role="Lateral Thinker",
    goal="You must find solutions out of the common schemes. You must reason with unusual ways of thinking but efficent",
    backstory="You are an expert in finding solutions out of nowhere to complex problem, you tend to reasonate in a way called out of the box and you bring to the solution an important factor that is the imprevidibility",
    allow_delegation=True,
    tool=[SerperDevTool()]
)

task = Task(
    description="Solve the enigma = im tall when im young, but im short when im older,what am i?",
    expected_output="a string containing the solution of the enigma and the whole process of thinking between the two agents",
    agent=logic
)



thinkers = Crew(
    agents=[logic,lathink],
    tasks=[task],
    process=Process.sequential,
    verbose=True
    
)

In [ ]:
result = thinkers.kickoff()


## Es 5 

In [17]:
from crewai import Agent,Task,Crew
from crewai_tools import SerperDevTool
from crewai import LLM,Process

ollama_llm= LLM(model="ollama/qwen3:0.6b",base_url="http://localhost:11434")
tool = SerperDevTool()

In [ ]:
researcher = Agent(
    role="Researcher",
    goal="Recollect some interesting facts about the {topic} given and pass them to your colleagues",
    backstory="You are a specialist in researching informations, you gather all reliable sources and you are an expert in summarizing everything if its too long",
    tools=[SerperDevTool()],
    verbose=True
    allow_delegation=True,
)

writer = Agent(
    role="Writer",
    goal="Write a 3 paragraph essay using the data given by your colleague",
    backstory="you are an excellent writer, you master the grammar really well and make no errors in logic and syntax, you are efficent and can even use complex words that fit the meaning of the whole paragraph. You do really well in writing long paragraphs too",
    tools=[SerperDevTool()]
    verbose=True,
    allow_delegation=True
)

editor = Agent(
    role="Editor",
    goal="you must take the paragraph given by your colleagues and publish it in a file named blog_post.txt",
    backstory="You are an esteemed publisher, you published thousands of works for your website",
    tools=[SerperDevTool()],
    verbose=True,
    allow_delegation=True
)


task_1 = Task(
    description="Recollect 3 or 4 interesting facts about the {topic} given",
    expected_output="A wall of text to give to your colleagues, without any errors",
    agent=researcher

)


task_2 = Task(
    description="Write a 2 or 3 paragraph essay using the material that your colleagues gave you",
    expected_output="2 or 3 paragraphs of text",
    agent=writer

)

task_3 = Task(
    description="take the paragraphs that your colleague wrote and save it in a file called blog_post.txt",
    expected_output="a saved file called blog_post.txt with the paragraphs of text in it",
    agent=editor
    output_file=

)



SyntaxError: invalid syntax. Perhaps you forgot a comma? (6661935.py, line 6)

## Es 6

In [ ]:
from crewai import Agent,Task,Crew
from crewai_tools import ScrapeWebsiteTool
from crewai import LLM,Process


In [ ]:
ollama_llm= LLM(model="ollama/qwen3:8b",base_url="http://localhost:11434")
tool = ScrapeWebsiteTool()
tool = ScrapeWebsiteTool(website_url=("https://www.laziodigital.it/artificial-intelligence-and-data-science-specialist/"))


In [ ]:
webagent = Agent(
    role="Web Scraper",
    goal="analyze and extract information requested by the user. The informations are contained in {info} and return them in a clear way without missing any word",
    backstory="You are a professional web scraper, like google you provide informations in the question asked, you are clear and fast",
    llm=ollama_llm,
    tools=[tool],
    embedder={
        "provider":"ollama", "url":"http://localhost:11434/api/embeddings",'model_name':"bge-m3"
    },
    verbose=True,

)

task = Task(
    description="you must analyze the {info} and scrape the website searching around the informations reguarding the requested information",
    expected_output="a clear text information extracted from the website",
    agent=webagent

)

crew = Crew(
    agents=[agent],
    tasks=[task]
    
)





In [ ]:
result = crew.kickoff()